In [ ]:
# Read the candidate csv file into a dataframe

import pandas as pd

CSV_PATH = "../results/NUMT_candidates.csv"

df = pd.read_csv(CSV_PATH)

In [ ]:
# Cluster reads that fall into the same bin

BIN_SIZE = 500

df["bin_start"] = (df["ref_start"] // BIN_SIZE) * BIN_SIZE

grouped = df.groupby(["ref_name", "bin_start", "is_reverse"])

clustered = grouped.agg(
    num_reads=("query_name", "count"),
    avg_ref_start=("ref_start", "mean"),
    avg_mate_start=("mate_start", "mean")
).reset_index()

143
